In [ ]:
''' PCA principal component analysis esta tecnica sirve para comprimir informacion de las imagenes manteniendo las caracteristicas mas importantes.
los datasets son grupos de datos que comparten ciertas caracteristicas, donde se extrae informacion clave 
 es dificil reinterpretar la informacion en pca
 variance is the sum of squares of differences between  all numbers and means
 es decir la varianza es la distancia  dispercion entre los datos y la media de los datos
 la covarianza  es la relacion de lasvariables entre si
 los eigen valores y eigenvectores son los vectores que definen la direccion de los datos y los valores que definen la magnitud de esos vectores del pca 
steps for PCA
1. standarize the data
2. calculate the covarianz matrix
3. compute the eigenvalues and eigenvectors
4. select principal components 
5. project data onto new basis
exercise:
feature example1 example2 example3 example4
 x          4         8         13     7
 y          11        4         5      14

x media = (4+8+13+7)/4=8.0
y media = (11+4+5+14)/4=8.5
feature example1 example2 example3 example4
 x          -4        0         5      -1
 y           2.5     -4.5      -3.5    5.5
la covarianza = (1/(n-1)*sum((xi-xmedia)(yi-ymean))

V1[0.55:0.83]  V2[-0.83:0.55]
eigv1 =30.38 eigv2=6.62

'''
